<a href="https://colab.research.google.com/github/hyeonji0401/CUDA_practice/blob/main/cudaMalloc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.1 CUDA 프로그램의 구조 및 흐름
호스트 코드가 반드시 필요한 이유

> CPU가 운영체제와 같은 컴퓨터 시스템의 기본 연산 장치이며, GPU와 같은 다른 연산 장치를 사용하기 위해서는 호스트 코드에서 커널을 호출해야 하기 때문임

* CPU와 GPU는 서로 독립된 장치로 사용하는 메모리 영역이 다름
* GPU는 디바이스 메모리를 사용하지만 모든 데이터는 기본적으로 호스트 메모리에 저장되어있음

=> GPU를 이용해서 데이터를 처리하기 위해서는 호스트 메모리에 있는 데이터를 디바이스 메모리로 복사해주어야함

<CUDA 프로그램 흐름>
1. 호스트 -> 디바이스 데이터 복사(GPU에서 처리할 데이터)
2. GPU연산
3. 디바이스 -> 호스트 데이터 복사(연산 결과 데이터)





# 3.2 CUDA 기초 메모리 API

**1. 디바이스 메모리 공간 할당 및 초기화**
> 디바이스 메모리로 데이터를 복사하기 위해서는 메모리에 사용할 공간을 할당받아야함

> 이는 C언어 malloc()함수와 동일함

> 차이점은 C언어는 호스트 공간의 메모리를 할당하는 것이고 cudaMalloc()은 디바이스 메모리 공간을 할당받는 것임

**디바이스 메모리 할당**
* cudaMalloc()

함수 원형


```
cudaError_t cudaMalloc(void **ptr, size_t size)
```
1. void **ptr
- 디바이스 메모리 공간의 시작 주소를 담을 포인터 변수의 주소

2. size_t size
- 할당할 공간의 크기(byte 단위)

3. 반환형
- cudaError_t 열거형




In [ ]:
%%cuda

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

int main(void)
{
    int *dDataPtr;
    cudaMalloc(&dDataPtr, sizeof(int)*32);
}


위의 코드는 디바이스 메모리 공간에 int형 데이터 32개를 담을 공간을 할당하는 예시임

위에서 설명했던 것과 같이 &dDatePtr가 할당된 메모리 공간의 시작주소가 저장되는 포인터 변수가 되는 것

* 주의할 점 : dDatePtr이 가리키는 주소는 디바이스 메모리 주소이므로 호스트 코드에서 직접 접근할 수 없다

그래서 만약


In [ ]:
%%cuda

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>

int main(void)
{
    int *dDataPtr;
    cudaMalloc(&dDataPtr, sizeof(int)*32);
    printf("%d", dDataPtr[0]);
    cudaFree(dDataPtr);
    return 0;
}


이와 같이 데이터에 접근하면 실행 오류가 발생함

**printf()는 CPU에서 작동하는 코드이기 때문임**



> CUDA 프로그램에서는 호스트 메모리 영역을 사용한느 변수와 디바이스 메모리 영역을 사용하는 변수를 구분하기 위해 일반적으로 디바이스 메모리 영역을 사용하는 변수의 이름 앞에 디바이스를 뜻하는 d를 붙여줌

* cudaFree()는 할당된 메모리를 해제하는 함수

**디바이스 메모리 해제**

* cudaFree()

함수 원형


```
cudaError_t cudaFree(void* ptr)
```
1.void *ptr
- 해제할 메모리 공간을 가리키는 포인터 변수

2. 반환형
- cudaError_t 열거형

**디바이스 메모리 초기화**

- cudaMemset()


> cudaMalloc()을 통해 메모리 공간을 할당받으면 해당 메모리 공간에 남아있던 쓰레기 값이 그대로 남아있으므로 초기화 해줘야함

함수 원형


```
cudaError_t cudaMemset(void *ptr, int value, size_t size)
```

1. void * ptr
- 값을 초기화할 메모리 공간의 시작 주소

2. int value
- 각 바이트를 초기화할 값

3. size_t size
- 초기화할 메모리 공간의 크기


**에러 코드 확인**

* cudaGetErrorName()


> CUDA API의 반환값 대부분은 에러코드(cudaError_t 열거형)이며 에러코드의 수는 그만큼 많음

> 각 에러코드 번호는 버전에 따라 변경될 수 있으며 모든 에러코드를 알 수는 없기 때문에 함수를 통해 에러의 종류를 확인하면 좋음

함수 원형


```
__host__ __device__ const char* cudaGetErrorName(cudaError_t error)
```

원형 앞에 __host__ 와 __device__ 가 모두 붙어 있는데, 이는 호스트와 디바이스 코드 모두에서 사용가능함을 말함


**디바이스 메모리 할당/초기화/해제 예제**



In [ ]:
%%cuda

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>

void checkDeviceMemory(void)
{
    size_t free, total;
    cudaMemGetInfo(&free, &total);
    printf("Device memory (free/total) = %lld/%lld bytes\n", free, total);
}

int main(void)
{
    int* dDataPtr;
    cudaError_t errorCode;

    checkDeviceMemory();
    errorCode = cudaMalloc(&dDataPtr, sizeof(int)*1024*1024);
    printf("cudaMalloc - %s\n", cudaGetErrorName(errorCode)); //메모리 할당 전
    checkDeviceMemory();

    errorCode = cudaMemset(dDataPtr, 0, sizeof(int)*1024*1024); //메모리 할당 후 가용 메모리 크기가 4MB 감소됨
    printf("cudaMalloc - %s\n", cudaGetErrorName(errorCode));

    errorCode = cudaFree(dDataPtr);
    printf("cudaFree - %s\n", cudaGetErrorName(errorCode)); //Free선언 후 다시 메모리 공간이 반환됨을 확인 할 수 있음
    checkDeviceMemory();
}

Device memory (free/total) = 15727656960/15835660288 bytes
cudaMalloc - cudaSuccess
Device memory (free/total) = 15723462656/15835660288 bytes
cudaMalloc - cudaSuccess
cudaFree - cudaSuccess
Device memory (free/total) = 15727656960/15835660288 bytes



cudaMalloc을 사용해서 int형 정수를 담을 수 있는 총 4MB의 공간을 할당하며, cudaMemset으로 모두 0으로 초기화하고 cudaFree로 메모리 공간을 해지하는 코드임


**[checkDeviceMemory()]**

디바이스 메모리의 총 크기와 현재 사용 가능한 공간의 크기를 반환해주는 함수로 cudaMemGetInfo()를 사용함

free: 현재 가용 메모리 크기

total: 사용하는 GPU가 가진 총 디바이스 메모리 크기

> 사용하는 GPU보다 큰 메모리를 할당할 경우 에러 발생함

**2. 호스트 - 디바이스 메모리 데이터 복사**



> 호스트 메모리와 디바이스 메모리는 서로 독립된 공간으로 다른 장치에 있는 데이터가 필요하다면 명시적인 데이터 복사 과정이 필요함

**장치 간 데이터 복사**

- cudaMemcpy()


> CUDA 프로그램에서 장치 간 데이터 복사를 위해 사용하는 함수

함수 원형


```
cudaError_t cudaMemcpy(void* dst, const void* src, size_t size, enum cudaMemcpyKind kind)
```
1. void* dist
- 복사될 메모리 공간의 시작 주소를 담고 있는 포인터 변수

2. const void* src
- 복사할 원본 데이터가 들어있는 메모리 공간의 시작 주소를 담고 있는 포인터 변수

3. size_t size
- 복사할 데이터의 크기(바이트 단위)

4. enum cudaMemcpyKind kind
- cudaMemcpyKind의 열거형 변수로 데이터 복사의 방향을 설정하는 인자
- 000ToXXX 형태로 000에서 XXX로 복사됨을 의미함
- 이 방향은 함수의 첫번째, 두번째 인자의 방향과 일치해야함

> cudaMemcpyDefault : dst와 src의 포인터 값에 의해 결정됨
- 이 경우 CUDA런타임이 방향성을 판단해서 복사를 수행해줌
- unified virtual addressing(호스트메모리와 디바이스 메모리를 하나의 공간처럼 가상화해줌)을 지원하는 시스템에서만 사용 가능
- 이 방법보다는 명시적으로 지정하는 것이 더 권장됨

<br>
TIP!

> CUDA 버전 6.0이상에서는 호스트 메모리와 디바이스 메모리를 하나의 논리적 주소 공간으로 사용할 수 있는 통합 메모리 기술을 지원한다. 이를 사용하면 명시적인 메모리 복사 없이 CUDA 프로그팸 작성가능하다. 그러나 높은 성능을 위해서는 명시적인 복사를 권장한다.



**장치 간 데이터 복사 예제**

In [ ]:
%%cuda
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>

__global__ void printData(int * _dDataPtr)
{
    printf("%d", _dDataPtr[threadIdx.x]);
}

__global__ void setData(int* _dDataPtr){  //배열의 모든 원소의 값을 2로 바꾸는 커널
    _dDataPtr[threadIdx.x]=2;
}

int main(void){

    int data[10]={0}; //호스트 메모리에 배열 만들기
    for(int i=0; i<10; i++) data[i] =1; //값 초기화

    int* dDataPtr;
    cudaMalloc(&dDataPtr, sizeof(int)*10); //디바이스 메모리 할당
    cudaMemset(dDataPtr, 0 ,sizeof(int)*10); //0으로 초기화

    printf("Data in device:");
    printData<<<1,10>>> (dDataPtr);

    cudaMemcpy(dDataPtr, data, sizeof(int)*10, cudaMemcpyHostToDevice); //호스트 메모리 -> 디바이스 메모리
    printf("\nHost->Device: ");
    printData<<<1,10>>> (dDataPtr);

    setData<<<1,10>>>(dDataPtr); //

    cudaMemcpy(data, dDataPtr, sizeof(int)*10, cudaMemcpyDeviceToHost); //디바이스 메모리 -> 호스트 메모리
    printf("\nDevice->Host: ");
    for(int i=0; i<10; i++) printf("%d", data[i]);

    cudaFree(dDataPtr);
    return 0;
}



Data in device:0000000000
Host->Device: 1111111111
Device->Host: 2222222222


# 3.3 CUDA로 작성하는 벡터의 합 프로그램

벡터 합을 구하는 호스트 프로그램

In [ ]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#define NUM_DATA 1024

int main(void){
    int *a, *b, *c;

    int memSize = sizeof(int)*NUM_DATA;
    a=new int[NUM_DATA]; memset(a, 0 ,memSize);
    b=new int[NUM_DATA]; memset(b, 0 ,memSize);
    c=new int[NUM_DATA]; memset(c, 0 ,memSize);

    for(int i=0; i<NUM_DATA; i++){
        a[i]= rand()%10;
        b[i]=rand()%10;
    }

    for(int i=0; i<NUM_DATA; i++){
        c[i] = a[i]+b[i];
    }

    delete[] a; delete[] b; delete[] c;
    return 0;

}

**"MEMSET"이란?**


> 메모리 블록을 특정 값으로 설정하는 데 사용되는 함수

함수 원형


```
void *memset(void *ptr, int value, size_t num);
```

- memset 함수는 메모리를 1바이트 단위로 초기화한다

잘못된 예시


```
memset(ptr, 1, 4) //4바이트를 1로 초기화
```
결과


```
00000001 00000001 00000001 00000001
```

> int 자료형은 하나당 4 bytes의 메모리 공간을 차지함

> 하지만 memset 함수는 1byte 단위로 c를 집어넣기 때문에 실제로 메모리에는 다음과 같은 수로 가득 차게 됨

> 그래서 대부분 0이나 Char 자료형으로 초기화 할때만 사용됨








**벡터 합을 GPU로 처리하려면?**

스켈레톤 코드



```
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

// The size of the vector
#define NUM_DATA 1024

// Simple vector sum kernel (Max vector size : 1024)
__global__ void vecAdd(int* _a, int* _b, int* _c) {
	int tID = threadIdx.x;
	_c[tID] = _a[tID] + _b[tID];
}

int main(void)
{
	int* a, * b, * c, * h_c;	// Vectors on the host
	int* d_a, * d_b, * d_c;		// Vectors on the device

	int memSize = sizeof(int) * NUM_DATA;
	printf("%d elements, memSize = %d bytes\n", NUM_DATA, memSize);

	// Memory allocation on the host-side
	a = new int[NUM_DATA]; memset(a, 0, memSize);
	b = new int[NUM_DATA]; memset(b, 0, memSize);
	c = new int[NUM_DATA]; memset(c, 0, memSize);
	h_c = new int[NUM_DATA]; memset(h_c, 0, memSize);

	// Data generation
	for (int i = 0; i < NUM_DATA; i++) {
		a[i] = rand() % 10;
		b[i] = rand() % 10;
	}

	// Vector sum on host (for performance comparision)
	for (int i = 0; i < NUM_DATA; i++)
		h_c[i] = a[i] + b[i];

	//****************************************//
	//******* Write your code - start ********//

	// 1. Memory allocation on the device-side (d_a, d_b, d_c)

	// 2. Data copy : Host (a, b) -> Device (d_a, d_b)

	// 3. Kernel call
	// vecAdd << <1, NUM_DATA >> > (d_a, d_b, d_c);

	// 4. Copy results : Device (d_c) -> Host (c)

	// 5. Release device memory (d_a, d_b, d_c)

	//******** Write your code - end *********//
	//****************************************//

	// Check results
	bool result = true;
	for (int i = 0; i < NUM_DATA; i++) {
		if (h_c[i] != c[i]) {
			printf("[%d] The resutl is not matched! (%d, %d)\n"
				, i, h_c[i], c[i]);
			result = false;
		}
	}

	if (result)
		printf("GPU works well!\n");

	// Release host memory
	delete[] a; delete[] b; delete[] c;

	return 0;
}

```

예제 정답

In [ ]:
%%cuda

# include "cuda_runtime.h"
# include "device_launch_parameters.h"

# include <stdio.h>
# include <stdlib.h>
# include <string.h>

// The size of the vector
# define NUM_DATA 1024

// Simple vector sum kernel (Max vector size : 1024)
__global__ void vecAdd(int* _a, int* _b, int* _c) {
    int tID = threadIdx.x;
    _c[tID] = _a[tID] + _b[tID];
}

int main(void)
{
    int* a, * b, * c, * h_c;    // Vectors on the host
    int* da, * db, * dc;        // Vectors on the device

    int memSize = sizeof(int) * NUM_DATA;
    printf("%d elements, memSize = %d bytes\n", NUM_DATA, memSize);

    // Memory allocation on the host-side
    a = new int[NUM_DATA]; memset(a, 0, memSize);
    b = new int[NUM_DATA]; memset(b, 0, memSize);
    c = new int[NUM_DATA]; memset(c, 0, memSize);
    h_c = new int[NUM_DATA]; memset(h_c, 0, memSize);

    // Data generation
    for (int i = 0; i < NUM_DATA; i++) {
        a[i] = rand() % 10;
        b[i] = rand() % 10;
    }

    // Vector sum on host (for performance comparision)
    for (int i = 0; i < NUM_DATA; i++)
        h_c[i] = a[i] + b[i];

    //****************************************//
    //******* Write your code - start ********//

    // 1. Memory allocation on the device-side (da, db, dc)
    cudaMalloc(&da, memSize); cudaMemset(da, 0, memSize); //할당 및 초기화
	  cudaMalloc(&db, memSize); cudaMemset(db, 0, memSize);
	  cudaMalloc(&dc, memSize); cudaMemset(dc, 0, memSize);

    // 2. Data copy : Host (a, b) -> Device (da, db)
    cudaMemcpy(da, a, memSize, cudaMemcpyHostToDevice);
    cudaMemcpy(db, b, memSize, cudaMemcpyHostToDevice);

    // 3. Kernel call
    vecAdd << <1, NUM_DATA >> > (da, db, dc);

    // 4. Copy results : Device (dc) -> Host (c)
    cudaMemcpy(c, dc, memSize, cudaMemcpyDeviceToHost);

    // 5. Release device memory (da, db, dc)
    cudaFree(da);
    cudaFree(db);
    cudaFree(dc);

    //******** Write your code - end *********//
    //****************************************//

    // Check results
    bool result = true;
    for (int i = 0; i < NUM_DATA; i++) {
        if (h_c[i] != c[i]) {
            printf("[%d] The resutl is not matched! (%d, %d)\n"
                , i, h_c[i], c[i]);
            result = false;
        }
    }

    if (result)
        printf("GPU works well!\n");

    // Release host memory
    delete[] a; delete[] b; delete[] c; //동적으로 할당된 배열 해제

    return 0;
}

1024 elements, memSize = 4096 bytes
GPU works well!



>  CUDA 프로그램을 비롯한 병렬 처리 알고리즘의 설계 및 구현은 디버그 난이도가 높으므로 작성 초반부터 호스트 코드와 같이 검증된 연산 결과와 비교하길 권장함

# 3.4 CUDA 알고리즘의 성능 측정

GPU를 이용해 병렬 처리를 수행하는 이유는 연산 성능 향상을 위함임

따라서 성능 측정이 필요함
<br>
<br>

**1. 커널 수행 시간**


> GPU연산은 커널 호출을 통해 진행되므로 CUDA 알고리즘 성능 측정을 위한 중요한 지점임


```
vecAdd<<<1, NUM_DATA>>> (da, db, dc); //커널 호출부
```
- 따라서 커널 호출 전에 시간 측정을 시작하고 커널 실행이 종료된 후 시간을 측정하면 커널 수행에 소요된 시간, 즉 GPU연산 시간을 측정할 수 있음

**주의할 점**
> 커널 호출 시 디바이스에게 명령을 전달한 후 프로그램 흐름의 제어권을 바로 호스트에게 반환한다는 점 즉, 호스트는 디바이스에게 커널 수행을 요청하고 바로 다음 작업을 진행함

> 이는 호스트와 디바이스의 비동기적 수행이 가능하도록하는 특징임

>따라서 디바이스가 수행 중인 작업이 끝날 때까지 대기하는 CUDA 동기화 함수 cudaDeviceSynchronize()를 커널 호출 다음에 넣고 시간 측정을 종료해야 커널의 수행 시간을 측정할 수 있음


**데이터 복사는 완료되고 커널이 수행되는 걸까?**


> 비동기적 특징이 있어도 CUDA API는 기본적으로 순차적으로 실행됨

> CUDA API는 스트림(stream)이라는 일종의 큐(선입선출 데이터 구조)를 통해 관리됨 따라서 호스트 코드를 디바이스 코드 제어에만 사용할 시 동기화함수 사용이 필수는 아님 그러나 모두를 이용해 연산하는 경우나 정보를 주고받을 때에는 필요함










**2. 데이터 전송 시간**


> 장치 간 데이터복사는 GPU를 이용하기 위해 발생하는 추가 작업으로 CUDA 알고리즘에서 발생하는 부하임 따라서 CUDA 알고리즘 성능 판단의 지표가 될 수 있음

> 호스트에서 디바이스로 디바이스에서 호스트로 메모리 복사하는 부분의 시작과 끝에 시간 측정 시작 및 종료를 해주면 확인할 수 있음

>cudaMemcpy는 호스트코드와 동기적으로 수행되므로 cudaDeviceSynchronize()함수가 필요하지 않음

예시


```
//시간 측정 시작
cudaMemcpy(da, a, memSize, cudaMemcpyHostToDevice);
//시간 측정 종료
```

- 이외에도 CUDA를 통해 GPU를 사용하기 위해 추가 작업이 있다면 해당 작업에 소요되는 시간도 포함해서 성능을 올바르게 확인할 수 있음




**CUDA 기반 벡터 합 프로그램의 성능 측정 및 분석**
> 교재에서는 DS_timer을 사용하는데 이는 폴더에 파일을 넣어줘야해서 다른 방식을 찾아봐야할 것 같음

< NVIDIA에서 권고하는 방식 >


```
// cudaEvent create
cudaEvent_t start, stop;
cudaEventCreate(&start);
cudaEventCreate(&stop);

. . .

cudaEventRecord(start, 0);

// 실행 시간을 측정할 GPU 코드

cudaEventRecord(stop, 0);
cudaEventSynchronize(stop);
float elapsedTime;
cudaEventElapsedTime(&elapsedTime, start, stop);

. . .

// cudaEvent destroy
cudaEventDestroy(start);
cudaEventDestroy(stop);
```

- GPU작업만 측정하므로 오버헤드가 적으며 높은 정확도를 보임
<br>
<br>

< 만약 CPU와 GPU 통합적으로 측정한다면? >

1. std::chrono

```
    auto start = std::chrono::high_resolution_clock::now();
    
    // CUDA 코드 실행
    
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end - start;

    std::cout << "Elapsed time: " << elapsed.count() << " seconds" << std::endl;
```
- 커널 실행 및 데이터 전송을 포함한 전체 프로그램의 실행 시간을 측정할 때 유용함
- 그러나 그만큼 동기화에 대해 주의를 기울여서 측정할 필요가 있음
- clock()은 초당 1,000번의 측정을 통해 1ms의 시간을 측정할 수 있음
- 단순한 프로그램 시간 측정에 적합
<br> <br>

2. QPC(QueryPerformanceCounter())


```
#include <stdio.h>
#include <Windows.h>
int main(void){

	LARGE_INTEGER ticksPerSec;
	LARGE_INTEGER start, end, diff;
    
	QueryPerformanceFrequency(&ticksPerSec);
	QueryPerformanceCounter(&start);
	// 시간을 측정하고 싶은 작업(예: 함수 호출)을 이곳에 삽입
    
	QueryPerformanceCounter(&end);
	// 측정값으로부터 실행시간 계산
	diff.QuadPart = end.QuadPart - start.QuadPart;
	printf("time: %.12f sec\n\n", ((double)diff.QuadPart/(double)ticksPerSec.QuadPart));
	return 0;
}
출처: https://udangtangtang-cording-oldcast1e.tistory.com/20 [우당탕탕 코딩 제작소:티스토리]
```
- 메인보드에 고해상도의 타이머가 존재하는데 이를 이용하여 특정 실행 시점들의 CPU 클럭수들을 얻어온 후 그 차이를 이용하여 작업 시간을 구함
- clock() 함수와 달리 1us 이하의 시간까지 측정
- 고정밀 프로그램 시간 측정에 적합 그러나 clock()과 마찬가지로 동기화에 주의를 기울여야함
- window환경에서만 사용가능함




**CUDA 기반 벡터 합 프로그램의 성능 측정**

host코드와 비교 측정을 위해 QPC 방법을 사용하고자 함

In [ ]:
%%cuda
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <windows.h>

// The size of the vector
#define NUM_DATA 1024

// Simple vector sum kernel (Max vector size : 1024)
__global__ void vecAdd(int* _a, int* _b, int* _c) {
	int tID = threadIdx.x;
	_c[tID] = _a[tID] + _b[tID];
}

int main(void)
{

	int* a, * b, * c, * hc;	// Vectors on the host
	int* da, * db, * dc;	// Vectors on the device

	int memSize = sizeof(int) * NUM_DATA;
	printf("%d elements, memSize = %d bytes\n", NUM_DATA, memSize);

	// Memory allocation on the host-side
	a = new int[NUM_DATA]; memset(a, 0, memSize);
	b = new int[NUM_DATA]; memset(b, 0, memSize);
	c = new int[NUM_DATA]; memset(c, 0, memSize);
	hc = new int[NUM_DATA]; memset(hc, 0, memSize);

	// Data generation
	for (int i = 0; i < NUM_DATA; i++) {
		a[i] = rand() % 10;
		b[i] = rand() % 10;
	}

  LARGE_INTEGER frequency, hostStart, hostEnd;
  double hostTime, totalTime, h2dTime, kernelTime, d2hTime;
  QueryPerformanceFrequency(&frequency);

	// Vector sum on host (for performance comparision)
  QueryPerformanceCounter(&hostStart);
	for (int i = 0; i < NUM_DATA; i++)
		hc[i] = a[i] + b[i];
  QueryPerformanceCounter(&hostEnd);
  hostTime = static_cast<double>(hostEnd.QuadPart - hostStart.QuadPart) / frequency.QuadPart * 1000;

	// Memory allocation on the device-side
	cudaMalloc(&da, memSize); cudaMemset(da, 0, memSize);
	cudaMalloc(&db, memSize); cudaMemset(db, 0, memSize);
	cudaMalloc(&dc, memSize); cudaMemset(dc, 0, memSize);

  LARGE_INTEGER deviceStart, deviceEnd;
  QueryPerformanceCounter(&deviceStart);

	// Data copy : Host -> Device
	LARGE_INTEGER h2dStart, h2dEnd;
	QueryPerformanceCounter(&h2dStart);
	cudaMemcpy(da, a, memSize, cudaMemcpyHostToDevice);
	cudaMemcpy(db, b, memSize, cudaMemcpyHostToDevice);
	QueryPerformanceCounter(&h2dEnd);
  h2dTime = static_cast<double>(h2dEnd.QuadPart - h2dEnd.QuadPart) / frequency.QuadPart * 1000;

	// Kernel call
	LARGE_INTEGER kernelStart, kernelEnd;
	QueryPerformanceCounter(&kernelStart);
	vecAdd <<<1, NUM_DATA >>> (da, db, dc);
	cudaDeviceSynchronize(); // synchronization function
  QueryPerformanceCounter(&endKernel);
  kernelTime = static_cast<double>(kernelEnd.QuadPart - kernelStart.QuadPart) / frequency.QuadPart * 1000;

	// Copy results : Device -> Host
  LARGE_INTEGER d2hStart, d2hEnd;
	QueryPerformanceCounter(&d2hStart);
	cudaMemcpy(c, dc, memSize, cudaMemcpyDeviceToHost);
  QueryPerformanceCounter(&d2hEnd);
  d2hTime = static_cast<double>(d2hEnd.QuadPart - d2hStart.QuadPart) / frequency.QuadPart * 1000;

	QueryPerformanceCounter(&deviceEnd);
  totalTime = static_cast<double>(deviceEnd.QuadPart - deviceStart.QuadPart) / frequency.QuadPart * 1000;

	// Release device memory
	cudaFree(da); cudaFree(db); cudaFree(dc);

  printf("Host Time: %f ms\n", hostTime);
  printf("Host -> Device: %f ms\n", h2dTime);
  printf("Kernel: %f ms\n", kernelTime);
  printf("Device -> Host: %f ms\n", d2hTime);
  printf("CUDA Total Time: %f ms\n", totalTime);

	// Check results
	bool result = true;
	for (int i = 0; i < NUM_DATA; i++) {
		if (hc[i] != c[i]) {
			printf("[%d] The result is not matched! (%d, %d)\n"
				, i, hc[i], c[i]);
			result = false;
		}
	}

	if (result)
		printf("GPU works well!\n");

	// Release host memory
	delete[] a; delete[] b; delete[] c;

	return 0;
}

/tmp/tmp8t2qwnxn/a177d02a-3b32-4da3-abab-149193fd245c/single_file.cu:7:10: fatal error: windows.h: No such file or directory
    7 | #include <windows.h>
      |          ^~~~~~~~~~~
compilation terminated.



작성은 호기롭게 다했는데 코랩에서는 window.h라이브러리가 안된다!!!!!!!

그래서 chrono 방식과 cudaEvent를 혼합해서 사용해야겠다

**chrono & cudaEvent**

In [3]:
%%cuda
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <chrono>
#include <iostream>

// The size of the vector
#define NUM_DATA 1024

// Simple vector sum kernel (Max vector size : 1024)
__global__ void vecAdd(int* _a, int* _b, int* _c) {
	int tID = threadIdx.x;
	_c[tID] = _a[tID] + _b[tID];
}

int main(void)
{

	int* a, * b, * c, * hc;	// Vectors on the host
	int* da, * db, * dc;	// Vectors on the device

	int memSize = sizeof(int) * NUM_DATA;
	printf("%d elements, memSize = %d bytes\n", NUM_DATA, memSize);

	// Memory allocation on the host-side
	a = new int[NUM_DATA]; memset(a, 0, memSize);
	b = new int[NUM_DATA]; memset(b, 0, memSize);
	c = new int[NUM_DATA]; memset(c, 0, memSize);
	hc = new int[NUM_DATA]; memset(hc, 0, memSize);

	// Data generation
	for (int i = 0; i < NUM_DATA; i++) {
		a[i] = rand() % 10;
		b[i] = rand() % 10;
	}


	// Vector sum on host (for performance comparision)
  auto start = std::chrono::high_resolution_clock::now();
	for (int i = 0; i < NUM_DATA; i++)
		hc[i] = a[i] + b[i];
  auto end = std::chrono::high_resolution_clock::now();
  // 밀리초 단위로 경과 시간 계산 (소수점 포함)
  std::chrono::duration<double, std::milli> elapsed = end - start;


  cudaEvent_t totalStart, totalStop, h2dStart, h2dStop, kernelStart, kernelStop, d2hStart, d2hStop;
  cudaEventCreate(&totalStart);
  cudaEventCreate(&totalStop);
  cudaEventCreate(&h2dStart);
  cudaEventCreate(&h2dStop);
  cudaEventCreate(&kernelStart);
  cudaEventCreate(&kernelStop);
  cudaEventCreate(&d2hStart);
  cudaEventCreate(&d2hStop);


	// Memory allocation on the device-side
	cudaMalloc(&da, memSize); cudaMemset(da, 0, memSize);
	cudaMalloc(&db, memSize); cudaMemset(db, 0, memSize);
	cudaMalloc(&dc, memSize); cudaMemset(dc, 0, memSize);

  cudaEventRecord(totalStart, 0);

	// Data copy : Host -> Device
	cudaEventRecord(h2dStart, 0);
	cudaMemcpy(da, a, memSize, cudaMemcpyHostToDevice);
	cudaMemcpy(db, b, memSize, cudaMemcpyHostToDevice);
	cudaEventRecord(h2dStop, 0);
  cudaEventSynchronize(h2dStop);
  float h2dTime;
  cudaEventElapsedTime(&h2dTime, h2dStart, h2dStop);

	// Kernel call
	cudaEventRecord(kernelStart, 0);
	vecAdd <<<1, NUM_DATA >>> (da, db, dc);
	cudaDeviceSynchronize(); // synchronization function
  cudaEventRecord(kernelStop, 0);
  cudaEventSynchronize(kernelStop);
  float kernelTime;
  cudaEventElapsedTime(&kernelTime, kernelStart, kernelStop);

	// Copy results : Device -> Host
  cudaEventRecord(d2hStart, 0);
	cudaMemcpy(c, dc, memSize, cudaMemcpyDeviceToHost);
  cudaEventRecord(d2hStop, 0);
  cudaEventSynchronize(d2hStop);
  float d2hTime;
  cudaEventElapsedTime(&d2hTime, d2hStart, d2hStop);

	cudaEventRecord(totalStop, 0);
  cudaEventSynchronize(totalStop);
  float totalTime;
  cudaEventElapsedTime(&totalTime, totalStart, totalStop);

	// Release device memory
	cudaFree(da); cudaFree(db); cudaFree(dc);

  // 결과 출력 (밀리초 단위, 소수점 포함)
  std::cout << "Host time: " << elapsed.count() << " ms" << std::endl;
  std::cout<<"Host -> Device: " << h2dTime << " ms" << std::endl;
  std::cout<<"Kernel: " << kernelTime << " ms" << std::endl;
  std::cout<<"Device -> Host: " << d2hTime << " ms" << std::endl;
  std::cout<<"CUDA Total Time: " << totalTime << " ms" << std::endl;

	// Check results
	bool result = true;
	for (int i = 0; i < NUM_DATA; i++) {
		if (hc[i] != c[i]) {
			printf("[%d] The result is not matched! (%d, %d)\n"
				, i, hc[i], c[i]);
			result = false;
		}
	}

	if (result)
		printf("GPU works well!\n");

	// Release host memory
	delete[] a; delete[] b; delete[] c;

	return 0;
}

1024 elements, memSize = 4096 bytes
Host time: 0.004024 ms
Host -> Device: 0.026624 ms
Kernel: 0.224288 ms
Device -> Host: 0.02 ms
CUDA Total Time: 0.301152 ms
GPU works well!



생각과는 다른 결과!!!

**CPU가 GPU보다 빠르다!**

이유:

> 1024의 벡터 크기는 GPU연산 성능을 발휘하기에는 턱없이 적은 숫자임
> 추가로 CUDA 함수 사용은 CUDA API 호출을 위한 기본 비용이 존재하기에 일정시간을 필요로 함

**그러면 벡터 크기를 늘린다면?**



In [6]:
%%cuda
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <chrono>
#include <iostream>

// The size of the vector
#define NUM_DATA 1048576

// Simple vector sum kernel (Max vector size : 1024)
__global__ void vecAdd(int* _a, int* _b, int* _c) {
	int tID = threadIdx.x;
	_c[tID] = _a[tID] + _b[tID];
}

int main(void)
{

	int* a, * b, * c, * hc;	// Vectors on the host
	int* da, * db, * dc;	// Vectors on the device

	int memSize = sizeof(int) * NUM_DATA;
	printf("%d elements, memSize = %d bytes\n", NUM_DATA, memSize);

	// Memory allocation on the host-side
	a = new int[NUM_DATA]; memset(a, 0, memSize);
	b = new int[NUM_DATA]; memset(b, 0, memSize);
	c = new int[NUM_DATA]; memset(c, 0, memSize);
	hc = new int[NUM_DATA]; memset(hc, 0, memSize);

	// Data generation
	for (int i = 0; i < NUM_DATA; i++) {
		a[i] = rand() % 10;
		b[i] = rand() % 10;
	}


	// Vector sum on host (for performance comparision)
  auto start = std::chrono::high_resolution_clock::now();
	for (int i = 0; i < NUM_DATA; i++)
		hc[i] = a[i] + b[i];
  auto end = std::chrono::high_resolution_clock::now();
  // 밀리초 단위로 경과 시간 계산 (소수점 포함)
  std::chrono::duration<double, std::milli> elapsed = end - start;


  cudaEvent_t totalStart, totalStop, h2dStart, h2dStop, kernelStart, kernelStop, d2hStart, d2hStop;
  cudaEventCreate(&totalStart);
  cudaEventCreate(&totalStop);
  cudaEventCreate(&h2dStart);
  cudaEventCreate(&h2dStop);
  cudaEventCreate(&kernelStart);
  cudaEventCreate(&kernelStop);
  cudaEventCreate(&d2hStart);
  cudaEventCreate(&d2hStop);


	// Memory allocation on the device-side
	cudaMalloc(&da, memSize); cudaMemset(da, 0, memSize);
	cudaMalloc(&db, memSize); cudaMemset(db, 0, memSize);
	cudaMalloc(&dc, memSize); cudaMemset(dc, 0, memSize);

  cudaEventRecord(totalStart, 0);

	// Data copy : Host -> Device
	cudaEventRecord(h2dStart, 0);
	cudaMemcpy(da, a, memSize, cudaMemcpyHostToDevice);
	cudaMemcpy(db, b, memSize, cudaMemcpyHostToDevice);
	cudaEventRecord(h2dStop, 0);
  cudaEventSynchronize(h2dStop);
  float h2dTime;
  cudaEventElapsedTime(&h2dTime, h2dStart, h2dStop);

	// Kernel call
	cudaEventRecord(kernelStart, 0);
	vecAdd <<<1, NUM_DATA >>> (da, db, dc);
	cudaDeviceSynchronize(); // synchronization function
  cudaEventRecord(kernelStop, 0);
  cudaEventSynchronize(kernelStop);
  float kernelTime;
  cudaEventElapsedTime(&kernelTime, kernelStart, kernelStop);

	// Copy results : Device -> Host
  cudaEventRecord(d2hStart, 0);
	cudaMemcpy(c, dc, memSize, cudaMemcpyDeviceToHost);
  cudaEventRecord(d2hStop, 0);
  cudaEventSynchronize(d2hStop);
  float d2hTime;
  cudaEventElapsedTime(&d2hTime, d2hStart, d2hStop);

	cudaEventRecord(totalStop, 0);
  cudaEventSynchronize(totalStop);
  float totalTime;
  cudaEventElapsedTime(&totalTime, totalStart, totalStop);

	// Release device memory
	cudaFree(da); cudaFree(db); cudaFree(dc);

  // 결과 출력 (밀리초 단위, 소수점 포함)
  std::cout << "Host time: " << elapsed.count() << " ms" << std::endl;
  std::cout<<"Host -> Device: " << h2dTime << " ms" << std::endl;
  std::cout<<"Kernel: " << kernelTime << " ms" << std::endl;
  std::cout<<"Device -> Host: " << d2hTime << " ms" << std::endl;
  std::cout<<"CUDA Total Time: " << totalTime << " ms" << std::endl;

	// Check results
	bool result = true;
	for (int i = 0; i < 1; i++) {
		if (hc[i] != c[i]) {
			printf("[%d] The result is not matched! (%d, %d)\n"
				, i, hc[i], c[i]);
			result = false;
		}
	}

	if (result)
		printf("GPU works well!\n");

	// Release host memory
	delete[] a; delete[] b; delete[] c;

	return 0;
}

1048576 elements, memSize = 4194304 bytes
Host time: 4.26018 ms
Host -> Device: 2.22157 ms
Kernel: 0.328736 ms
Device -> Host: 1.13136 ms
CUDA Total Time: 3.71702 ms
[0] The result is not matched! (9, 0)



속도 확인을 위해 결과 출력은 1만 일단 해보았음(출력이 너무 길어서 코랩이 삭제해버린다,,,)

결과:  
>CUDA 알고리즘이 속도 면에서는 더 빨라졌지만
대부분 연산 결과가 일치하지 않는 다는 메시지가 출력되었음

이는 CUDA 스레드 계층 구조 및 실행 모델의 이해가 필요함

4장에서 계속,,,,,
